## Jupyter - LAMMPS notebook
### _simulation and post-processing_


In [ ]:
from lammps import IPyLammps
import numpy as np

In [ ]:
# Initialize LAMMPS
lmp = IPyLammps()

# Set up the simulation box
x = 10
y = 10
z = 10
rho = 0.5
ps = 0.168
t = 1.8071
rc = 3.5
p = 200
lmp.variable("s","equal", 10)
d = p * "$s"

lmp.units("lj")
lmp.atom_style("atomic")

lmp.lattice("fcc", rho)
lmp.region("box", "block", 0, x, 0, y, 0, z)
lmp.create_box(1, "box")
lmp.create_atoms(1, "box")
lmp.mass(1, 1.0)

lmp.compute("thermo_t", "all temp")
lmp.compute("1", "all pressure thermo_t")

lmp.velocity("all create", f"{t} 87287")

lmp.pair_style("lj/cut", rc)
lmp.pair_coeff(1, 1, 1.0, 1.0)

lmp.neighbor(0.3, "bin")
lmp.neigh_modify("delay", 0, "every", 1)

# 1st NVT run
lmp.fix("1", "all npt", f"temp {t} {t} 100.0 iso {ps} {ps} 100.0")

lmp.thermo(1000)
lmp.thermo_style("custom", "step temp press")
lmp.run(10000)

lmp.velocity("all scale", t)

# Thermal conductivity calculation
lmp.reset_timestep(0)

lmp.compute("myKE", "all ke/atom")
lmp.compute("myPE", "all pe/atom")
lmp.compute("myStress", "all stress/atom NULL virial")
lmp.compute("flux", "all heat/flux myKE myPE myStress")
lmp.variable("Jx", "equal c_flux[1]/vol")
lmp.variable("Jy", "equal c_flux[2]/vol")
lmp.variable("Jz", "equal c_flux[3]/vol")

# For thermal conductivity
lmp.fix("JJ", "all ave/correlate", "10 200 2000", "c_flux[1] c_flux[2] c_flux[3] type auto", "file JJ_TC.dat ave running")

lmp.variable("scale", "equal $(10*dt)/1.8071/1.8071/vol")
lmp.variable("k11", "equal trap(f_JJ[3])*${scale}")
lmp.variable("k22", "equal trap(f_JJ[4])*${scale}")
lmp.variable("k33", "equal trap(f_JJ[5])*${scale}")

# For GK- viscosity
lmp.variable("pxy", "equal pxy")
lmp.variable("pxz", "equal pxz")
lmp.variable("pyz", "equal pyz")

lmp.fix("SS", "all ave/correlate", "10 200 2000", "v_pxy v_pxz v_pyz type auto file SS_visc.dat ave running")

lmp.compute("myRDF", "all rdf 100")
lmp.fix("rdf1", "all ave/time", "1 1000 1000", "c_myRDF[*] file rdf.txt mode vector")

lmp.variable("vscale", "equal  $(10*dt)*(10*10*10)/1.8071")
lmp.variable("v11", "equal trap(f_SS[3])*${vscale}")
lmp.variable("v22", "equal trap(f_SS[4])*${vscale}")
lmp.variable("v33", "equal trap(f_SS[5])*${vscale}")

lmp.thermo(2000)
lmp.thermo_style("custom", "step temp press v_k11 v_k22 v_k33 v_v11 v_v22 v_v33")

lmp.run(10000)

lmp.variable("kappa", "equal 1000*0.187972*(v_k11+v_k22+v_k33)/3.0")
                            


# eta from LJ to Pas
# eta = (eta_star) * sqrt(m * å)/ó^2  LJ units
# for Argon:
# multiply by 9.07*10-5 Pa*s
# multiply by 907 ugr/cm*s

lmp.variable("v", "equal 907*(v_v11+v_v22+v_v33)/3.0")


In [ ]:
lmp.print("Running_average_conductivity:" "${kappa}")

In [ ]:
lmp.print("average_viscosity:" "${v}")